In [2]:
import os
import json
import spotipy
import pandas as pd
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
from json.decoder import JSONDecodeError

# Replace 'your-app-username' with your Spotify username
username = 'karmafiend'
scope = 'user-read-private user-read-playback-state user-modify-playback-state'

# Ensure you have set SPOTIPY_CLIENT_ID and SPOTIPY_CLIENT_SECRET as environment variables
client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = 'https://www.google.com/'

# Set up the Spotify OAuth object
sp_oauth = SpotifyOAuth(client_id=client_id, 
                        client_secret=client_secret, 
                        redirect_uri=redirect_uri, 
                        scope=scope, 
                        cache_path=".spotipy_cache")

# Try to get a valid token for the user, this will automatically refresh the token if it's expired
token_info = sp_oauth.get_cached_token()

if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(f"Please navigate here in your browser: {auth_url}")
    response = input("Enter the URL you were redirected to: ")
    code = sp_oauth.parse_response_code(response)
    token_info = sp_oauth.get_access_token(code)

# Create a Spotify client with the access token
sp = spotipy.Spotify(auth=token_info['access_token'])

# Function to get playlist details
def get_playlist_details(playlist_id):
    results = sp.playlist(playlist_id)
    return results

# Example usage: replace 'your-playlist-id' with the playlist ID you want to get details for
playlist_id = '0Hm1tCeFv45CJkNeIAtrfF'
playlist_details = get_playlist_details(playlist_id)
print(playlist_details)


Please navigate here in your browser: https://accounts.spotify.com/authorize?client_id=46776cd8abee49d9a182ecfaf6707dea&response_type=code&redirect_uri=https%3A%2F%2Fwww.google.com%2F&scope=user-read-private+user-read-playback-state+user-modify-playback-state


Enter the URL you were redirected to:  https://www.google.com/?code=AQCiA9hgWIi8qZ11koOMm3ydpMm7MMkWWzE1JqT0816Uw_5bs0iRgISwBNitq26TgEwf0bg5N1mUCPiT4v8j7OgBiXMpMnjYrx2xK4JMzN7hpn0AAnL8rTPRrBzNuZWdzccmKcGNvwQgcwlJuYz-Zhp0Zgjh-uqMdnw4nbu0H9ev3CgLiSnqeTKNeKL2OPR-j6Hd5MWfIsD4ASArnDaN1UOgIzNWaQXrdCbJW7PsIwR8sulq2I0HjnnXn70Ko1LM3xb8Yw


/var/folders/25/3j8rh09j3tnb3725bp132lbh0000gn/T/ipykernel_22743/1354578974.py:33: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


{'collaborative': False, 'description': 'A list of the top 100 artists on Spotify based on monthly listenership. Last updated: 17 March 2024', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0Hm1tCeFv45CJkNeIAtrfF'}, 'followers': {'href': None, 'total': 6948}, 'href': 'https://api.spotify.com/v1/playlists/0Hm1tCeFv45CJkNeIAtrfF?additional_types=track', 'id': '0Hm1tCeFv45CJkNeIAtrfF', 'images': [{'height': 640, 'url': 'https://mosaic.scdn.co/640/ab67616d00001e028b58d20f1b77295730db15b4ab67616d00001e02bef074de9ca825bddaeb9f46ab67616d00001e02c87bfeef81a210ddb7f717b5ab67616d00001e02e787cffec20aa2a396a61647', 'width': 640}, {'height': 300, 'url': 'https://mosaic.scdn.co/300/ab67616d00001e028b58d20f1b77295730db15b4ab67616d00001e02bef074de9ca825bddaeb9f46ab67616d00001e02c87bfeef81a210ddb7f717b5ab67616d00001e02e787cffec20aa2a396a61647', 'width': 300}, {'height': 60, 'url': 'https://mosaic.scdn.co/60/ab67616d00001e028b58d20f1b77295730db15b4ab67616d00001e02bef074de9ca825bddaeb9f4

In [7]:
#Let's build a dictionary, dataframe and export a CSV. First, initialize an empty list to store the extracted data
tracks_data = []

#Create a for loop to cycle through the API output collecting the info we want. Then loop through each track in the playlist
for i, item in enumerate(playlist_details['tracks']['items'], start=1):
    track = item['track']
    # Each track can have multiple artists; concatenate their names and IDs
    artist_names = ', '.join([artist['name'] for artist in track['artists']]) #since multiple artists can appear on a track, concatenate them
    artist_ids = ', '.join([artist['id'] for artist in track['artists']]) #similar as above with ID
    available_markets = ', '.join(track['available_markets']) #list all available markets for analysis
    
    #Append the data for the current track to our list, in effect, filling up the list with the desired api call output
    tracks_data.append({
        'Ranking': i,
        'Track Name': track['name'],
        'Artist Names': artist_names,
        'Artist IDs': artist_ids,
        'Available Markets': available_markets
    })

#Convert the list of dictionaries to a DataFrame
df_tracks = pd.DataFrame(tracks_data)

#Define the CSV file name
csv_file_name = 'spotify_top100_playlist_tracks.csv'

#Export the DataFrame to a CSV file
df_tracks.to_csv(csv_file_name, index=False)

print(f"Data exported to {csv_file_name}")


Data exported to spotify_top100_playlist_tracks.csv
